In [9]:
import warnings
from urllib3.exceptions import NotOpenSSLWarning

warnings.filterwarnings("ignore", category=NotOpenSSLWarning)

In [10]:
import spacy
spacy_nlp = spacy.load("en_core_web_sm")
import utility_functions as utils
import importlib
import pandas as pd
import seaborn as sns

# Text processing
from gensim import corpora

# Visualization
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
from tqdm import tqdm
from gensim import models
from gensim.models.coherencemodel import CoherenceModel

importlib.reload(utils)

data = './preprocessed_df.pkl'

In [11]:
df = pd.read_pickle(data)
df[['Artist', 'Song', 'Tokens', 'Processed_Lyrics', 'Lyrics', 'Coast']].head()

,Artist,Song,Tokens,Processed_Lyrics,Lyrics,Coast
0,Big L,8 Iz Enuff.mp3,"[crew, house, terra, herb, mcgruff, buddah, bl...",crew house terra herb mcgruff buddah bless big...,"\n\nYo\nMy crew is in the house\nTerra, Herb M...",east_coast
1,Big L,Da Graveyard.mp3,"[number, one, crew, area, big, lightin, nigga,...",number one crew area big lightin nigga incense...,\n\n[Intro:]\nIt's the number one crew in the ...,east_coast
2,Big L,I Don't Understand It.mp3,"[many, mcs, overrated, ask, even, supposed, ma...",many mcs overrated ask even supposed make rap ...,\n\n[Verse 1]\nThere are too many MC's who are...,east_coast
3,Big L,"No Endz, No Skinz.mp3","[let, get, point, real, quick, pockets, thick,...",let get point real quick pockets thick mad chi...,\n\n[Verse 1:]\nLet me get to the point real q...,east_coast
4,Big L,MVP.mp3,"[spark, phillies, pass, stout, making, quick, ...",spark phillies pass stout making quick money g...,\n\n[Big L]\n A yo spark up the phillies and p...,east_coast


In [12]:
df['Lyrics'] = df['Lyrics'].apply(utils.light_preprocessing)

In [13]:
list_of_unpreprocessed_documents = df['Lyrics'].tolist()
list_of_preprocessed_documents = df['Tokens'].apply(lambda tokens: ' '.join(tokens)).tolist()

In [14]:
import itertools
import torch
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from tqdm import tqdm
from gensim.models import CoherenceModel
import pandas as pd

# Define hyperparameters to search over
num_topics_range = [5, 10, 15, 20]  # Different number of topics
learning_rates = [0.002, 0.01, 0.05]  # Learning rates
epochs_range = [20, 50]  # Number of epochs

# Create all possible combinations of hyperparameters
hyperparameters_combinations = list(itertools.product(num_topics_range, learning_rates, epochs_range))

# Prepare topic model data
tp = TopicModelDataPreparation("all-mpnet-base-v2")
training_dataset = tp.fit(text_for_contextual=list_of_unpreprocessed_documents, text_for_bow=list_of_preprocessed_documents)

# Define a list to store coherence values
coherence_results = []

# Perform hyperparameter search
for num_topics, learning_rate, num_epochs in tqdm(hyperparameters_combinations, desc="Hyperparameter Search"):
    # Train CombinedTM model
    ctm_model = CombinedTM(
        bow_size=len(tp.vocab),
        contextual_size=768,
        n_components=num_topics,
        num_epochs=num_epochs,
        lr=learning_rate,
    )

    ctm_model.fit(training_dataset)

    # Get the topic-word distribution
    topics = ctm_model.get_topic_lists(10)

    # Prepare for coherence calculation (based on BoW representation)
    bow_docs = [doc.split() for doc in list_of_preprocessed_documents]

    # Calculate coherence using Gensim's CoherenceModel
    coherence_model = CoherenceModel(topics=topics, texts=bow_docs, dictionary=tp.vocab, coherence='c_v')
    coherence_score = coherence_model.get_coherence()

    # Store the results
    coherence_results.append({
        'num_topics': num_topics,
        'learning_rate': learning_rate,
        'num_epochs': num_epochs,
        'coherence_score': coherence_score
    })
results_df = pd.DataFrame(coherence_results)

# Print the best hyperparameter combination based on coherence score
best_params = results_df.loc[results_df['coherence_score'].idxmax()]
print("Best Parameters based on Coherence Score:")
print(best_params)

KeyboardInterrupt: 

In [24]:
results_df

NameError: name 'results_df' is not defined

In [23]:
# Plot the coherence scores for comparison
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.lineplot(data=results_df, x="num_topics", y="coherence_score", hue="learning_rate", style="num_epochs", markers=True)
plt.xlabel("Number of Topics")
plt.ylabel("Coherence Score (c_v)")
plt.title("Coherence Scores for Different CombinedTM Hyperparameters")
plt.legend(title="Learning Rate & Epochs")
plt.show()

NameError: name 'results_df' is not defined

<Figure size 1000x600 with 0 Axes>

In [27]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation

# Initialize data preparation
qt = TopicModelDataPreparation("all-mpnet-base-v2")

# Prepare training dataset using both versions
training_dataset = qt.fit(text_for_contextual=list_of_unpreprocessed_documents, text_for_bow=list_of_preprocessed_documents)

# Initialize and train CombinedTM
ctm = CombinedTM(bow_size=len(qt.vocab), contextual_size=768, n_components=2)  # 50 topics
ctm.fit(training_dataset)



/Users/borosabel/lib/python3.9/site-packages/contextualized_topic_models/utils/data_preparation.py:64: UserWarning: the longest document in your collection has 1919 words, the model instead truncates to 128 tokens.
  warnings.warn(
/Users/borosabel/lib/python3.9/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/Users/borosabel/lib/python3.9/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/Users/borosabel/lib/python3.9/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creat

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

0it [00:00, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingfac

defaultdict(<class 'list'>, {0: ['as', 'dream'], 1: ['stitch', 'deliver']})


In [29]:
# Get the topics
topics = ctm.get_topics(10)
print(topics)

defaultdict(<class 'list'>, {0: ['as', 'dream', 'let', 'kind', 'would', 'time', 'feel', 'keep', 'get', 'go'], 1: ['stitch', 'deliver', 'commit', 'criticize', 'rally', 'junction', 'shakur', 'bell', 'violence', 'burnin']})
